In [17]:
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.calibration import LabelEncoder

In [18]:
# -------------------------------
# 1. Load dataset
# -------------------------------
def load_dataset(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    print(df.info())
    return df

In [19]:
# -------------------------------
# 2. Handle missing values
# -------------------------------
def fill_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    num_cols = df.select_dtypes(include=['float64', 'int64']).columns
    cat_cols = df.select_dtypes(include=['object', 'category', 'bool']).columns

    # Numerical → median
    df[num_cols] = df[num_cols].fillna(df[num_cols].median())

    # Categorical → mode
    for col in cat_cols:
        mode = df[col].mode()
        df[col] = df[col].fillna(mode[0] if not mode.empty else 'Unknown')

    return df

In [20]:
# -------------------------------
# 3. Scale numerical features
# -------------------------------
def scale_numerical(df: pd.DataFrame) -> pd.DataFrame:
    df_scaled = df.copy()
    num_cols = df_scaled.select_dtypes(include=['float64', 'int64']).columns
    scaler = StandardScaler()
    df_scaled[num_cols] = scaler.fit_transform(df_scaled[num_cols])
    return df_scaled, scaler

In [21]:
# -------------------------------
# 4. Encode categorical features
# -------------------------------

def encode_categorical(df: pd.DataFrame):
    df_encoded = df.copy()
    cat_cols = df_encoded.select_dtypes(include=['object', 'category', 'bool']).columns

    le_dict = {}
    for col in cat_cols:
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df_encoded[col].fillna('NaN_Label'))
        le_dict[col] = le
    
    return df_encoded, le_dict

In [22]:
# -------------------------------
# 5. Feature selection (correlation + importance)
# -------------------------------
def select_features(df_encoded: pd.DataFrame, target_col: str, N: int = 20):
    from sklearn.ensemble import RandomForestClassifier

    X = df_encoded.drop(columns=[target_col])
    y = df_encoded[target_col]

    X = pd.get_dummies(X)  # one-hot if needed
    X_train, _, y_train, _ = train_test_split(X, y, random_state=42)

    # Correlation
    corr_with_target = df_encoded.corr(numeric_only=True)[target_col].drop(target_col).abs()
    top_corr = corr_with_target.sort_values(ascending=False).head(N).index.tolist()

    # Random Forest importance
    rf = RandomForestClassifier(random_state=42)
    rf.fit(X_train, y_train)
    importances = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
    top_importance = importances.head(N).index.tolist()

    # Combine
    selected_features = list(set(top_corr + top_importance))
    return selected_features

In [23]:
# -------------------------------
# 6. Build preprocessed dataframe
# -------------------------------
def build_preprocessed_df(df_encoded: pd.DataFrame, selected_features: list, target_col: str):
    X = pd.get_dummies(df_encoded.drop(columns=[target_col]))
    valuable_features_df = X[selected_features]

    # Add target
    preprocessed_df = valuable_features_df.copy()
    preprocessed_df[target_col] = df_encoded[target_col]
    return preprocessed_df

In [24]:
# -------------------------------
# 7. Downsample classes
# -------------------------------
def downsample_classes(df: pd.DataFrame, target_col: str):
    from sklearn.utils import resample

    df_downsample = df.copy()
    class_counts = df_downsample[target_col].value_counts()
    min_count = class_counts.min()

    dfs = []
    for cls in class_counts.index:
        cls_df = df_downsample[df_downsample[target_col] == cls]
        cls_downsampled = resample(cls_df, replace=False, n_samples=min_count, random_state=42)
        dfs.append(cls_downsampled)

    df_downsampled = pd.concat(dfs).sample(frac=1, random_state=42).reset_index(drop=True)
    return df_downsampled

In [25]:
path = "Data/cumulative_2025.10.03_07.59.03.csv"
target_col = "koi_disposition"

df = load_dataset(path)
df_filled = fill_missing_values(df)
df_scaled, scaler = scale_numerical(df_filled)
df_encoded, le_dict = encode_categorical(df_scaled)

selected_features = select_features(df_encoded, target_col, N=20)
preprocessed_df = build_preprocessed_df(df_encoded, selected_features, target_col)

df_downsampled = downsample_classes(preprocessed_df, target_col)

print("Final preprocessed dataset shape:", df_downsampled.shape)
print(df_downsampled.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9564 entries, 0 to 9563
Data columns (total 50 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   loc_rowid          9564 non-null   int64  
 1   kepid              9564 non-null   int64  
 2   kepoi_name         9564 non-null   object 
 3   kepler_name        2747 non-null   object 
 4   koi_disposition    9564 non-null   object 
 5   koi_pdisposition   9564 non-null   object 
 6   koi_score          8054 non-null   float64
 7   koi_fpflag_nt      9564 non-null   int64  
 8   koi_fpflag_ss      9564 non-null   int64  
 9   koi_fpflag_co      9564 non-null   int64  
 10  koi_fpflag_ec      9564 non-null   int64  
 11  koi_period         9564 non-null   float64
 12  koi_period_err1    9110 non-null   float64
 13  koi_period_err2    9110 non-null   float64
 14  koi_time0bk        9564 non-null   float64
 15  koi_time0bk_err1   9110 non-null   float64
 16  koi_time0bk_err2   9110 

c:\Users\Victus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\Victus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\Victus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


Final preprocessed dataset shape: (5937, 30)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5937 entries, 0 to 5936
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   koi_slogg_err1     5937 non-null   float64
 1   koi_prad_err1      5937 non-null   float64
 2   loc_rowid          5937 non-null   float64
 3   koi_teq            5937 non-null   float64
 4   koi_period_err1    5937 non-null   float64
 5   koi_model_snr      5937 non-null   float64
 6   koi_duration       5937 non-null   float64
 7   dec                5937 non-null   float64
 8   koi_score          5937 non-null   float64
 9   koi_fpflag_co      5937 non-null   float64
 10  koi_depth          5937 non-null   float64
 11  koi_duration_err1  5937 non-null   float64
 12  koi_tce_delivname  5937 non-null   int32  
 13  ra                 5937 non-null   float64
 14  kepoi_name         5937 non-null   int32  
 15  koi_steff_err2     5937 non

In [26]:
df = df_downsampled.drop(columns=['koi_pdisposition']).copy()

In [27]:
x_train, x_test, y_train, y_test = train_test_split(
    df.drop(columns=[target_col]),  # Features
    df[target_col],                 # Target
    test_size=0.2,
    random_state=42,
    stratify=df[target_col]
)

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Fit a Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(x_train, y_train)

# Predict on test set
y_pred = rf.predict(x_test)

# Print accuracy
print("Test accuracy:", accuracy_score(y_test, y_pred))


Test accuracy: 0.9949494949494949


In [29]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import recall_score, roc_auc_score
from sklearn.preprocessing import label_binarize

# Assume you have these (replace with your actual data)
# X_train: np.array or torch.Tensor, shape [n_samples, input_size]
# y_train: np.array or torch.Tensor, shape [n_samples], values 0-2
# X_test, y_test: similar

# Convert to tensors if not already
X_train = torch.tensor(x_train.to_numpy(), dtype=torch.float32)
y_train = torch.tensor(y_train.to_numpy(), dtype=torch.long)
X_test = torch.tensor(x_test.to_numpy(), dtype=torch.float32)
y_test = torch.tensor(y_test.to_numpy(), dtype=torch.long)

# Create datasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Data loaders
batch_size = 32  # Adjust based on your data size and memory
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Compute class weights for imbalance (optional but recommended for recall)
class_counts = np.bincount(y_train.numpy())
class_weights = 1. / class_counts
class_weights = torch.tensor(class_weights / class_weights.sum(), dtype=torch.float32)  # Normalize

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    def __init__(self, in_features, out_features, dropout_rate=0.3):  # <-- fixed __init__
        super(ResidualBlock, self).__init__()  # <-- fixed __init__
        self.fc = nn.Linear(in_features, out_features)
        self.ln = nn.LayerNorm(out_features)  # Layer norm for better stability
        self.dropout = nn.Dropout(dropout_rate)
        
        # Projection if dimensions differ
        self.proj = nn.Linear(in_features, out_features) if in_features != out_features else nn.Identity()
    
    def forward(self, x):
        residual = self.proj(x)
        x = F.silu(self.ln(self.fc(x)))  # Swish activation
        x = self.dropout(x)
        return x + residual  # Skip connection


class ResidualMLP(nn.Module):
    def __init__(self, input_size=16, num_classes=3):  # <-- fixed __init__
        super(ResidualMLP, self).__init__()  # <-- fixed __init__
        self.entry = nn.Linear(input_size, 256)  # Entry layer
        
        # Residual blocks for depth
        self.res_block1 = ResidualBlock(256, 256)
        self.res_block2 = ResidualBlock(256, 128)
        self.res_block3 = ResidualBlock(128, 128)
        self.res_block4 = ResidualBlock(128, 64)
        
        self.dropout_final = nn.Dropout(0.2)
        self.fc_out = nn.Linear(64, num_classes)
    
    def forward(self, x):
        x = F.silu(self.entry(x))
        
        x = self.res_block1(x)
        x = self.res_block2(x)
        x = self.res_block3(x)
        x = self.res_block4(x)
        
        x = self.dropout_final(x)
        x = self.fc_out(x)  # Logits output
        return x

In [31]:
print("CUDA Available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    print("Number of GPUs:", torch.cuda.device_count())
    print("Current Device Index:", torch.cuda.current_device())

CUDA Available: True
GPU Name: NVIDIA GeForce RTX 4070 Laptop GPU
Number of GPUs: 1
Current Device Index: 0


In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

import torch
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# Ensure y_train is numpy
if isinstance(y_train, torch.Tensor):
    y_train_np = y_train.cpu().numpy()
else:
    y_train_np = np.array(y_train)

# Get unique classes
classes = np.unique(y_train_np)

# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train_np
)

# Convert to tensor for PyTorch
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

print("Classes:", classes)
print("Class Weights:", class_weights)


# ---- Focal Loss with class weights ----
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha  # can be class weights tensor
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none', weight=self.alpha)
        pt = torch.exp(-ce_loss)  # probability of true class
        focal_loss = (1 - pt) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        return focal_loss


# ---- Initialize with class weights ----
loss_fn = FocalLoss(alpha=class_weights, gamma=2.0)


Classes: [0 1 2]
Class Weights: tensor([1., 1., 1.], device='cuda:0')


In [33]:
%pip install torch
import torch
import torch.nn as nn
import numpy as np

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize model
input_size = X_train.shape[1]  # number of features
model = ResidualMLP(input_size=input_size, num_classes=3).to(device)

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=0.005, weight_decay=1e-5)

# Learning rate scheduler (ReduceLROnPlateau equivalent)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.8, patience=6, min_lr=1e-6
)

# Early stopping setup
patience = 20
best_accuracy = 0.0
epochs_no_improve = 0
best_model_path = "best_residual_mlp_model.pth"

# Training loop
epochs = 300
for epoch in range(epochs):
    # --- Training ---
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_X)
        
        loss = loss_fn(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()
    
    avg_train_loss = train_loss / len(train_loader)
    train_accuracy = 100 * correct / total

    # --- Evaluation on test set ---
    model.eval()
    test_loss = 0.0
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            loss = loss_fn(outputs, batch_y)
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            total_test += batch_y.size(0)
            correct_test += (predicted == batch_y).sum().item()
    
    avg_test_loss = test_loss / len(test_loader)
    test_accuracy = 100 * correct_test / total_test

    # --- Scheduler step ---
    old_lr = optimizer.param_groups[0]['lr']
    scheduler.step(test_accuracy)
    new_lr = optimizer.param_groups[0]['lr']
    if new_lr < old_lr:
        print(f"📉 Learning rate reduced from {old_lr:.6f} to {new_lr:.6f}")

    # --- Save best model ---
    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        torch.save(model.state_dict(), best_model_path)
        print(f"✅ Saved new best model at epoch {epoch+1} with Test Accuracy: {best_accuracy:.2f}%")
        epochs_no_improve = 0  # reset patience counter
    else:
        epochs_no_improve += 1

    # --- Logging ---
    print(
        f"Epoch {epoch+1}/{epochs} | "
        f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}% | "
        f"Test Loss: {avg_test_loss:.4f}, Test Acc: {test_accuracy:.2f}%"
    )


print(f"\n🎯 Training finished. Best Test Accuracy: {best_accuracy:.2f}%")


Note: you may need to restart the kernel to use updated packages.
✅ Saved new best model at epoch 1 with Test Accuracy: 66.50%
Epoch 1/300 | Train Loss: 14.0493, Train Acc: 64.96% | Test Loss: 0.2204, Test Acc: 66.50%
✅ Saved new best model at epoch 2 with Test Accuracy: 73.06%
Epoch 2/300 | Train Loss: 1.2330, Train Acc: 65.66% | Test Loss: 0.1140, Test Acc: 73.06%
Epoch 3/300 | Train Loss: 0.1353, Train Acc: 72.25% | Test Loss: 0.1135, Test Acc: 71.04%
Epoch 4/300 | Train Loss: 0.2811, Train Acc: 76.67% | Test Loss: 0.2373, Test Acc: 65.57%
✅ Saved new best model at epoch 5 with Test Accuracy: 87.54%
Epoch 5/300 | Train Loss: 0.1635, Train Acc: 74.79% | Test Loss: 0.1017, Test Acc: 87.54%
Epoch 6/300 | Train Loss: 0.1175, Train Acc: 81.43% | Test Loss: 0.1846, Test Acc: 67.76%
Epoch 7/300 | Train Loss: 0.0992, Train Acc: 83.36% | Test Loss: 0.1021, Test Acc: 76.35%
Epoch 8/300 | Train Loss: 0.0997, Train Acc: 83.70% | Test Loss: 0.3142, Test Acc: 69.95%
✅ Saved new best model at epoc

In [34]:
from sklearn.metrics import classification_report

# --- Evaluate on test set and generate classification report ---
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = model(batch_X)
        _, predicted = torch.max(outputs, 1)
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(batch_y.cpu().numpy())

# Generate report
print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))



📊 Classification Report:
              precision    recall  f1-score   support

           0     0.9801    0.9975    0.9887       396
           1     1.0000    1.0000    1.0000       396
           2     0.9974    0.9798    0.9885       396

    accuracy                         0.9924      1188
   macro avg     0.9925    0.9924    0.9924      1188
weighted avg     0.9925    0.9924    0.9924      1188

